We use Keras-Tuner to tune EfficientNet model. Only image data is used. 

In [ ]:
!pip install -q tensorflow==2.3.0 # Use 2.3.0 for built-in EfficientNet
!pip install -q git+https://github.com/keras-team/keras-tuner@master # Use github head for newly added TPU support
!pip install -q cloud-tpu-client # Needed for sync TPU version

In [ ]:
import random, re, math
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf, tensorflow.keras.backend as K
from kaggle_datasets import KaggleDatasets
print('Tensorflow version ' + tf.__version__)
import kerastuner as kt

Initiate TPU

In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
    # Sync TPU version
    from cloud_tpu_client import Client
    c = Client()
    c.configure_tpu_version(tf.__version__, restart_type='ifNeeded')
    
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None
    

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
from tensorflow.data.experimental import AUTOTUNE

# Configuration
IMAGE_SIZE = [384, 384]
EPOCHS = 15
BATCH_SIZE = 16 * strategy.num_replicas_in_sync

# Data access
GCS_PATH = KaggleDatasets().get_gcs_path('melanoma-384x384')

# training filenames directory
TRAINING_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/train*.tfrec')
# test filenames directory
TEST_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/test*.tfrec')
# submission file
submission = pd.read_csv('/kaggle/input/siim-isic-melanoma-classification/sample_submission.csv')

Augmentation for TPU as KPL not supporting yet

In [ ]:
def get_mat(rotation, shear, height_zoom, width_zoom, height_shift, width_shift):
    # returns 3x3 transformmatrix which transforms indicies
        
    # CONVERT DEGREES TO RADIANS
    rotation = math.pi * rotation / 180.
    shear = math.pi * shear / 180.
    
    # ROTATION MATRIX
    c1 = tf.math.cos(rotation)
    s1 = tf.math.sin(rotation)
    one = tf.constant([1],dtype='float32')
    zero = tf.constant([0],dtype='float32')
    rotation_matrix = tf.reshape( tf.concat([c1,s1,zero, -s1,c1,zero, zero,zero,one],axis=0),[3,3] )
        
    # SHEAR MATRIX
    c2 = tf.math.cos(shear)
    s2 = tf.math.sin(shear)
    shear_matrix = tf.reshape( tf.concat([one,s2,zero, zero,c2,zero, zero,zero,one],axis=0),[3,3] )    
    
    # ZOOM MATRIX
    zoom_matrix = tf.reshape( tf.concat([one/height_zoom,zero,zero, zero,one/width_zoom,zero, zero,zero,one],axis=0),[3,3] )
    
    # SHIFT MATRIX
    shift_matrix = tf.reshape( tf.concat([one,zero,height_shift, zero,one,width_shift, zero,zero,one],axis=0),[3,3] )
    
    return K.dot(K.dot(rotation_matrix, shear_matrix), K.dot(zoom_matrix, shift_matrix))

def transform(image, label):
    # input image - is one image of size [dim,dim,3] not a batch of [b,dim,dim,3]
    # output - image randomly rotated, sheared, zoomed, and shifted
    DIM = IMAGE_SIZE[0]
    XDIM = DIM%2 #fix for size 331
    
    if 0.5 > tf.random.uniform([1], minval = 0, maxval = 1):
        rot = 15. * tf.random.normal([1],dtype='float32')
    else:
        rot = 180. * tf.random.normal([1],dtype='float32')
    if 0.5 > tf.random.uniform([1], minval = 0, maxval = 1):
        shr = 5. * tf.random.normal([1],dtype='float32') 
    else:
        shr = 2. * tf.random.normal([1],dtype='float32')
    if 0.5 > tf.random.uniform([1], minval = 0, maxval = 1):
        h_zoom = 1.0 + tf.random.normal([1],dtype='float32')/10. 
    else:
        h_zoom = 1.0 + tf.random.normal([1],dtype='float32')/8.
    if 0.5 > tf.random.uniform([1], minval = 0, maxval = 1):
        w_zoom = 1.0 + tf.random.normal([1],dtype='float32')/10. 
    else:
        w_zoom = 1.0 + tf.random.normal([1],dtype='float32')/8.
    if 0.5 > tf.random.uniform([1], minval = 0, maxval = 1):
        h_shift = 16. * tf.random.normal([1],dtype='float32') 
    else:
        h_shift = 8. * tf.random.normal([1],dtype='float32')
    if 0.5 > tf.random.uniform([1], minval = 0, maxval = 1):
        w_shift = 16. * tf.random.normal([1],dtype='float32') 
    else:
        w_shift = 8. * tf.random.normal([1],dtype='float32')
  
    # GET TRANSFORMATION MATRIX
    m = get_mat(rot,shr,h_zoom,w_zoom,h_shift,w_shift) 

    # LIST DESTINATION PIXEL INDICES
    x = tf.repeat( tf.range(DIM//2,-DIM//2,-1), DIM )
    y = tf.tile( tf.range(-DIM//2,DIM//2),[DIM] )
    z = tf.ones([DIM*DIM],dtype='int32')
    idx = tf.stack( [x,y,z] )
    
    # ROTATE DESTINATION PIXELS ONTO ORIGIN PIXELS
    idx2 = K.dot(m,tf.cast(idx,dtype='float32'))
    idx2 = K.cast(idx2,dtype='int32')
    idx2 = K.clip(idx2,-DIM//2+XDIM+1,DIM//2)
    
    # FIND ORIGIN PIXEL VALUES           
    idx3 = tf.stack( [DIM//2-idx2[0,], DIM//2-1+idx2[1,]] )
    d = tf.gather_nd(image,tf.transpose(idx3))
        
    return image, label

# function to decode our images (normalize and reshape)
def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)
    # convert image to floats in [0, 255] range
    image = tf.cast(image, tf.float32)
    # explicit size needed for TPU
    image = tf.ensure_shape(image, [*IMAGE_SIZE, 3])
    return image

# this function parse our images and also get the target variable
def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        # tf.string means bytestring
        "image": tf.io.FixedLenFeature([], tf.string), 
        # shape [] means single element
        "target": tf.io.FixedLenFeature([], tf.int64),
        # meta features
        "age_approx": tf.io.FixedLenFeature([], tf.int64),
        "sex": tf.io.FixedLenFeature([], tf.int64),
        "anatom_site_general_challenge": tf.io.FixedLenFeature([], tf.int64)
        
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    label = tf.cast(example['target'], tf.int32)
    # returns a dataset of (image, label)
    return image, label

# this function parse our image and also get our image_name (id) to perform predictions
def read_unlabeled_tfrecord(example):
    UNLABELED_TFREC_FORMAT = {
        # tf.string means bytestring
        "image": tf.io.FixedLenFeature([], tf.string), 
        # shape [] means single element
        "image_name": tf.io.FixedLenFeature([], tf.string),
        # meta features
        "age_approx": tf.io.FixedLenFeature([], tf.int64),
        "sex": tf.io.FixedLenFeature([], tf.int64),
        "anatom_site_general_challenge": tf.io.FixedLenFeature([], tf.int64)
    }
    example = tf.io.parse_single_example(example, UNLABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    image_name = example['image_name']
    # returns a dataset of (image, key)
    return image, image_name
    
def load_dataset(filenames, labeled = True, ordered = False):
    # Read from TFRecords. For optimal performance, reading from multiple files at once and
    # Diregarding data order. Order does not matter since we will be shuffling the data anyway
    
    ignore_order = tf.data.Options()
    if not ordered:
        # disable order, increase speed
        ignore_order.experimental_deterministic = False 
        
    # automatically interleaves reads from multiple files
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads = AUTOTUNE)
    # use data as soon as it streams in, rather than in its original order
    dataset = dataset.with_options(ignore_order)
    # returns a dataset of (image, label) pairs if labeled = True or (image, id) pair if labeld = False
    dataset = dataset.map(read_labeled_tfrecord if labeled else read_unlabeled_tfrecord, num_parallel_calls = AUTOTUNE) 
    return dataset


def data_augment(image, label):
    # data augmentation. Thanks to the dataset.prefetch(AUTO) statement 
    # in the next function (below), this happens essentially for free on TPU. 
    # Data pipeline code is executed on the "CPU" part
    # of the TPU while the TPU itself is computing gradients.
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_hue(image, 0.01)
    image = tf.image.random_saturation(image, 0.7, 1.3)
    image = tf.image.random_contrast(image, 0.8, 1.2)
    image = tf.image.random_brightness(image, 0.1)
    
    return image, label

def get_training_dataset(filenames, labeled = True, ordered = False):
    dataset = load_dataset(filenames, labeled = labeled, ordered = ordered)
    dataset = dataset.map(data_augment, num_parallel_calls = AUTOTUNE)
    dataset = dataset.map(transform, num_parallel_calls = AUTOTUNE)
    # the training dataset must repeat for several epochs
    dataset = dataset.repeat()
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
    # prefetch next batch while training (autotune prefetch buffer size)
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset

def get_validation_dataset(filenames, labeled = True, ordered = True):
    dataset = load_dataset(filenames, labeled = labeled, ordered = ordered)
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
    # using gpu, not enought memory to use cache
    # dataset = dataset.cache()
    # prefetch next batch while training (autotune prefetch buffer size)
    dataset = dataset.prefetch(AUTOTUNE) 
    return dataset

def get_test_dataset(filenames, labeled = False, ordered = True):
    dataset = load_dataset(filenames, labeled = labeled, ordered = ordered)
    dataset = dataset.batch(BATCH_SIZE)
    # prefetch next batch while training (autotune prefetch buffer size)
    dataset = dataset.prefetch(AUTOTUNE) 
    return dataset

# function to count how many photos we have in
def count_data_items(filenames):
    # the number of data items is written in the name of the .tfrec files, i.e. flowers00-230.tfrec = 230 data items
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

# this function parse our images and also get the target variable
def read_tfrecord_full(example):
    LABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), 
        "image_name": tf.io.FixedLenFeature([], tf.string), 
        "target": tf.io.FixedLenFeature([], tf.int64), 
        # meta features
        "age_approx": tf.io.FixedLenFeature([], tf.int64),
        "sex": tf.io.FixedLenFeature([], tf.int64),
        "anatom_site_general_challenge": tf.io.FixedLenFeature([], tf.int64)
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    image_name = example['image_name']
    target = tf.cast(example['target'], tf.float32)
    # meta features
    data = {}
    data['age_approx'] = tf.cast(example['age_approx'], tf.int32)
    data['sex'] = tf.cast(example['sex'], tf.int32)
    data['anatom_site_general_challenge'] = tf.cast(tf.one_hot(example['anatom_site_general_challenge'], 7), tf.int32)
    return image, image_name, target, data

def load_dataset_full(filenames):        
    # automatically interleaves reads from multiple files
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads = AUTOTUNE)
    # returns a dataset of (image_name, target)
    dataset = dataset.map(read_tfrecord_full, num_parallel_calls = AUTOTUNE) 
    return dataset

def get_data_full(filenames):
    dataset = load_dataset_full(filenames)
    dataset = dataset.map(setup_input3, num_parallel_calls = AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset





In [ ]:
def binary_focal_loss(gamma=2., alpha=.25):
    """
    Binary form of focal loss.
      FL(p_t) = -alpha * (1 - p_t)**gamma * log(p_t)
      where p = sigmoid(x), p_t = p or 1 - p depending on if the label is 1 or 0, respectively.
    References:
        https://arxiv.org/pdf/1708.02002.pdf
    Usage:
     model.compile(loss=[binary_focal_loss(alpha=.25, gamma=2)], metrics=["accuracy"], optimizer=adam)
    """
    def binary_focal_loss_fixed(y_true, y_pred):
        """
        :param y_true: A tensor of the same shape as `y_pred`
        :param y_pred:  A tensor resulting from a sigmoid
        :return: Output tensor.
        """
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))

        epsilon = K.epsilon()
        # clip to prevent NaN's and Inf's
        pt_1 = K.clip(pt_1, epsilon, 1. - epsilon)
        pt_0 = K.clip(pt_0, epsilon, 1. - epsilon)

        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1)) \
               -K.sum((1 - alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0))

    return binary_focal_loss_fixed


    


In [ ]:
import sklearn
TRAIN_FILES, VAL_FILES = sklearn.model_selection.train_test_split(TRAINING_FILENAMES, train_size=0.8)
train_dataset = get_training_dataset(TRAIN_FILES, labeled = True, ordered = False)
val_dataset = get_validation_dataset(VAL_FILES, labeled = True, ordered = False)
all_dataset = get_training_dataset(TRAIN_FILES + VAL_FILES, labeled=True, ordered=False)
K.clear_session()


In [ ]:
for x, y in train_dataset.take(1):
    print(x.numpy().shape, y)

In [ ]:
NUM_TRAINING_IMAGES = count_data_items(TRAIN_FILES)
# use validation data for training
NUM_VALIDATION_IMAGES = count_data_items(VAL_FILES)
NUM_TEST_IMAGES = count_data_items(TEST_FILENAMES)
STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE

print('Dataset: {} training images, {} validation images, {} unlabeled test images'.format(NUM_TRAINING_IMAGES, NUM_VALIDATION_IMAGES, NUM_TEST_IMAGES))

The following is some not-yet-merged function. 

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras

from tensorflow.keras import layers
from tensorflow.keras.applications import efficientnet
from tensorflow.keras.layers.experimental import preprocessing

from kerastuner.engine import hypermodel

import os


EFFICIENTNET_MODELS = {'B0': efficientnet.EfficientNetB0,
                       'B1': efficientnet.EfficientNetB1,
                       'B2': efficientnet.EfficientNetB2,
                       'B3': efficientnet.EfficientNetB3,
                       'B4': efficientnet.EfficientNetB4,
                       'B5': efficientnet.EfficientNetB5,
                       'B6': efficientnet.EfficientNetB6,
                       'B7': efficientnet.EfficientNetB7}

EFFICIENTNET_IMG_SIZE = {'B0': 224,
                         'B1': 240,
                         'B2': 260,
                         'B3': 300,
                         'B4': 380,
                         'B5': 456,
                         'B6': 528,
                         'B7': 600}


class HyperEfficientNet(hypermodel.HyperModel):
    """An EfficientNet HyperModel.
    Models built by this HyperModel takes input image data in
    ints [0, 255]. The output data should be one-hot encoded
    with number of classes matching `classes`.
      # Arguments:
        include_top: whether to include the fully-connected
            layer at the top of the network. Model is not
            compiled if include_top is set to False.
        input_shape: shape tuple, e.g. `(256, 256, 3)`.
              Input images will be resized if different from
              the default input size of the version of
              efficientnet base model used.
              One of `input_shape` or `input_tensor` must be
              specified.
        input_tensor: Keras tensor to use as image input for the model.
              One of `input_shape` or `input_tensor` must be
              specified.
        classes: number of classes to classify images into.
        weights: str or None. Default is 'imagenet', where the weights pre-trained
              on imagenet will be downloaded. Otherwise the weights will be
              loaded from the directory in 'weights', and are expected to be in
              h5 format with naming convention '{weights}/b{n}_notop.h5' where n
              is 0 to 7. If set to None, the weights will be initiated from scratch.
        augmentation_model: optional Model or HyperModel for image augmentation.
        **kwargs: Additional keyword arguments that apply to all
            HyperModels. See `kerastuner.HyperModel`.
    """
    def __init__(self,
                 include_top=True,
                 input_shape=None,
                 input_tensor=None,
                 classes=None,
                 weights='imagenet',
                 augmentation_model=None,
                 **kwargs):
        if not isinstance(augmentation_model, (hypermodel.HyperModel,
                                               keras.Model,
                                               type(None))):
            raise ValueError('Keyword augmentation_model should be '
                             'a HyperModel, a Keras Model or empty. '
                             'Received {}.'.format(augmentation_model))

        if include_top and classes is None:
            raise ValueError('You must specify `classes` when '
                             '`include_top=True`')

        if input_shape is None and input_tensor is None:
            raise ValueError('You must specify either `input_shape` '
                             'or `input_tensor`.')

        self.include_top = include_top
        self.input_shape = input_shape
        self.input_tensor = input_tensor
        self.classes = classes
        self.augmentation_model = augmentation_model
        self.weights = weights

        super(HyperEfficientNet, self).__init__(**kwargs)

    def build(self, hp):

        if self.input_tensor is not None:
            inputs = tf.keras.utils.get_source_inputs(self.input_tensor)
            x = self.input_tensor
        else:
            inputs = layers.Input(shape=self.input_shape)
            x = inputs

        if self.augmentation_model:
            if isinstance(self.augmentation_model, hypermodel.HyperModel):
                augmentation_model = self.augmentation_model.build(hp)
            elif isinstance(self.augmentation_model, keras.models.Model):
                augmentation_model = self.augmentation_model

            x = augmentation_model(x)

        # Select one of pre-trained EfficientNet as feature extractor
        version = hp.Choice('version',
                            ['B{}'.format(i) for i in range(8)],
                            default='B0')
        img_size = EFFICIENTNET_IMG_SIZE[version]

        weights = self.weights
        if weights and (weights != 'imagenet'):
            weights = os.path.join(weights, version.lower())
            weights += '_notop.h5'
            if not os.path.isfile(weights):
                raise ValueError('Expect path {} to include weight file; but '
                                 'no file is found'.format(weights))

        x = preprocessing.Resizing(img_size, img_size, interpolation='bilinear')(x)
        efficientnet_model = EFFICIENTNET_MODELS[version](include_top=False,
                                                          input_tensor=x,
                                                          weights=weights)

        # Rebuild top layers of the model.
        x = efficientnet_model.output

        pooling = hp.Choice('pooling', ['avg', 'max'], default='avg')
        if pooling == 'avg':
            x = layers.GlobalAveragePooling2D(name='avg_pool')(x)
        elif pooling == 'max':
            x = layers.GlobalMaxPooling2D(name='max_pool')(x)

        if self.include_top:
            top_dropout_rate = hp.Float('top_dropout_rate',
                                        min_value=0.2,
                                        max_value=0.8,
                                        default=0.2)
            x = layers.Dropout(top_dropout_rate, name='top_dropout')(x)

            x = layers.Dense(
                self.classes, activation='sigmoid', name='probs')(x)

            # compile
            model = keras.Model(inputs, x, name='EfficientNet')
            self._compile(model, hp)

            return model
        else:
            return keras.Model(inputs, x, name='EfficientNet')

    def _compile(self, model, hp):
        """ Compile model using hyperparameters in hp.
            When subclassing the hypermodel, this may
            be overriden to change behavior of compiling.
        """
        learning_rate = hp.Choice('learning_rate', [0.1, 0.01, 0.001], default=0.01)
        optimizer = tf.keras.optimizers.SGD(
                momentum=0.1,
                learning_rate=learning_rate)

        model.compile(
            optimizer=optimizer,
            loss='categorical_crossentropy',
            metrics=['accuracy'],
            experimental_steps_per_execution=4)


In [ ]:
# Define HyperModel using built-in application
# from kerastuner.applications.efficientnet import HyperEfficientNet
hm = HyperEfficientNet(input_shape=[IMAGE_SIZE[0], IMAGE_SIZE[1], 3] , classes=1)


# Optional: Restrict default hyperparameters.
# To take effect, pass this `hp` instance when constructing tuner as `hyperparameters=hp`
from kerastuner.engine.hyperparameters import HyperParameters
hp = HyperParameters()
hp.Choice('version', ['B0', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6']) #restrict choice of EfficientNet version from B0-B7 to B0-B4

# Initiate Tuner
tuner = kt.tuners.randomsearch.RandomSearch(
    hypermodel=hm,
    objective=kt.Objective("val_auc", direction="max"),
    max_trials=7,
    hyperparameters=hp,
    distribution_strategy=strategy, # This strategy's scope is used for building each model during the search.
    directory='tuner_melanoma',
    project_name='hyperband_efficientnet',
    overwrite=True,
    loss='binary_crossentropy',
    metrics=['AUC']
)
tuner.search_space_summary()

In [ ]:
tuner.search(train_dataset,
             epochs=15,
             validation_data=val_dataset,
             steps_per_epoch=STEPS_PER_EPOCH,
             verbose=2)

In [ ]:
model = tuner.get_best_models()[0]
model.fit(all_dataset, epochs=40, steps_per_epoch=(NUM_VALIDATION_IMAGES + NUM_TRAINING_IMAGES)//BATCH_SIZE, callbacks=[tf.keras.callbacks.ReduceLROnPlateau(monitor='AUC')])


In [ ]:
test_dataset = get_test_dataset(TEST_FILENAMES)
targets = []
image_names = []
np_decode = np.vectorize(lambda x: x.decode('UTF-8'))
for image, image_name in test_dataset:
    targets.append(model(image))
    image_names.append(np_decode(image_name.numpy()))

In [ ]:
targets = np.concatenate(targets).squeeze()
image_names = np.concatenate(image_names)

In [ ]:
submission = pd.DataFrame(dict(
    image_name = image_names,
    target = targets
))

submission = submission.sort_values('image_name') 
submission.to_csv('submission.csv', index=False)

In [ ]:
submission.head()

In [ ]:
submission['target']